# Quest: Newsroom Editor

    - Goal: Craft briefs from offline news fixtures
- Goal: Require approvals for opinionated pieces

    **Pass criteria:** `briefs.jsonl` holds briefs with word counts and approvals.

    _Mock mode uses fixtures and writes artifacts under `_runs/`. Flip `USE_REAL_CLIENT=True` when ready for live calls._

In [ ]:

USE_REAL_CLIENT = False  # flip when ready
import os, json, time, pathlib, math, random

RUN = pathlib.Path('_runs/quest_newsroom_editor')
RUN.mkdir(parents=True, exist_ok=True)
FIXTURE_ROOT = pathlib.Path(os.environ.get('MOCK_FIXTURE_ROOT', '_fixtures'))

def write_meta(**kwargs):
    payload = {"timestamp": time.time(), **kwargs}
    (RUN / 'meta.json').write_text(json.dumps(payload, indent=2))
    return payload

USE_REAL_CLIENT = bool(str(os.environ.get('USE_REAL_CLIENT', USE_REAL_CLIENT)).lower() in {'1', 'true', 'yes'})


In [ ]:
approval_fixture = json.loads((FIXTURE_ROOT / "approvals.json").read_text())
                def build_brief(article):
                    base_sentences = [
                        f"Headline: {article['title']}",
                        article["summary"],
                        article["body"],
                        "The piece explains why the topic matters for local stakeholders.",
                        "Analysts highlight lessons for agent workflows.",
                    ]
                    text = " ".join(base_sentences)
                    filler = "This briefing remains grounded in verified fixture data."
                    while len(text.split()) < 125:
                        text += " " + filler
                    text = " ".join(text.split()[:150])
                    citations = [article["citations"][0], {"source": "MockArchive", "url": "https://fixtures.local/mock"}]
                    approval_info = {"required": article["opinionated"], "status": "approved" if article["opinionated"] else "not-requested"}
                    if article["opinionated"]:
                        approval_info["approved_by"] = approval_fixture["approved_by"]
                    return {
                        "slug": article["slug"],
                        "brief": text,
                        "word_count": len(text.split()),
                        "citations": citations,
                        "approval": approval_info,
                    }
                briefs = []
                for path in sorted((FIXTURE_ROOT / "news").glob("sample_*.json"))[:4]:
                    article = json.loads(path.read_text())
                    briefs.append(build_brief(article))
                output_path = RUN / "briefs.jsonl"
                with output_path.open("w", encoding="utf-8") as handle:
                    for entry in briefs:
                        handle.write(json.dumps(entry) + "
")
                [entry["word_count"] for entry in briefs]

In [ ]:
opinionated = sum(1 for entry in briefs if entry['approval']['required'])
meta = write_meta(ok=True, lab="quest_newsroom_editor", briefs=len(briefs), opinionated=opinionated)
meta